# Step #1 Azure OpenAI Service - Chat on private data using LangChain

Import note this tutorial is for langchain 0.2
The functions below might not work with other langchain versions

In [45]:
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Step #2 Define an LLM Config
First, we will simply invoke the LLM with a basic prompt.


In [46]:
# Init LLM and embeddings model
llm = AzureChatOpenAI(deployment_name="gpt4o", 
                      temperature=0.5,
                      max_tokens=100, 
                      openai_api_version="2023-03-15-preview")

response = llm.invoke("Tell me a joke")
print(response.content)

Sure, here's one for you:

Why don't scientists trust atoms?

Because they make up everything!


# Config LLM
Next we will determine the behavior of our llms configs. This allows us to reuse them for different purposes.

In [48]:
# Define configuration with system instructions
config_writer = {
    "deployment_name": "gpt4o",
    "temperature": 0.6,
    "max_tokens": 300,
    "openai_api_version": "2023-03-15-preview",
    "instruction": "You are a creative writing assistant."
}

# Init LLM model
llm_writer = AzureChatOpenAI(deployment_name=config_writer["deployment_name"], 
                      temperature=config_writer["temperature"],
                      max_tokens=config_writer["max_tokens"], 
                      openai_api_version=config_writer["openai_api_version"])

# Add system instruction to the prompt
user_prompt = 'Write an application letter for a job as a software engineer.'
response = llm_writer.invoke(f'{config_writer["instruction"]} ' + user_prompt)
content = response.content
print(content)

[Your Name]  
[Your Address]  
[City, State, ZIP Code]  
[Email Address]  
[Phone Number]  
[Date]

[Recipient's Name]  
[Job Title]  
[Company Name]  
[Company Address]  
[City, State, ZIP Code]

Dear [Recipient's Name],

I am writing to express my enthusiastic interest in the Software Engineer position at [Company Name], as advertised on [where you found the job posting]. With a strong foundation in computer science, a passion for innovative problem-solving, and hands-on experience in software development, I am excited about the opportunity to contribute to your team and help drive the next wave of technological advancements at [Company Name].

I hold a [Your Degree] in Computer Science from [Your University], where I developed a robust understanding of algorithms, data structures, and software design principles. My academic background is complemented by professional experience at [Previous Company Name], where I worked as a [Your Previous Job Title]. In this role, I was responsible 

In [49]:
config_reviewer = {
    "deployment_name": "gpt4o",
    "temperature": 0.4,
    "max_tokens": 500,
    "openai_api_version": "2023-03-15-preview",
    "instruction": "You are a critical reviewer, who's an expert in reviewing others content and providing helpful advice for improvements." 
}

# Init LLM model
llm_params = {key: config_reviewer[key] for key in config_reviewer if key != "instruction"}  # Exclude 'instruction'
llm_reviewer = AzureChatOpenAI(**llm_params)

user_prompt = 'Review the following content: ' + content
response = llm_reviewer.invoke(f'{config_reviewer["instruction"]} ' + user_prompt)
content = response.content
print(content)

letter continues below:

technical proficiency spans multiple programming languages, including Java, Python, and C++, as well as experience with frameworks such as Django and Spring. I am also well-versed in agile methodologies and have a proven track record of delivering high-quality software solutions on time and within budget.

What excites me most about the Software Engineer position at [Company Name] is the opportunity to work on cutting-edge projects that push the boundaries of technology. I am particularly impressed by [specific project or aspect of the company that excites you], and I am eager to bring my skills and experience to your team to help achieve the company's goals.

I am confident that my background and passion for software engineering make me a strong candidate for this position. I look forward to the opportunity to discuss how my skills and experiences align with the needs of your team. Thank you for considering my application. I hope to hear from you soon to arran

# Step #3 Agents

In [50]:
# The core idea of agents is to use a language model to choose a sequence of actions to take. 
# In chains, a sequence of actions is hardcoded (in code). 
# In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
# More information: https://python.langchain.com/v0.1/docs/modules/agents/

In [51]:
from langchain_community.llms import OpenLLM
from langchain.chains import SimpleChain
from langchain.memory import Memory
from langchain.agents import Agent

# Load configuration from file
config = LangChainConfig.from_file("langchain.yaml")

# Initialize components based on the configuration
memory = Memory.from_config(config.memory)
simple_chain = SimpleChain.from_config(config.chains['simple_chain'])
llm = OpenLLM(
    model_name=config.model_name,
    model_id=config.model_id,
    server_url=config.server_url if hasattr(config, 'server_url') else None
)
agent = Agent.from_config(config.agents['default_agent'], memory, llm=llm)

# Run the agent
agent.run()

ImportError: cannot import name 'SimpleChain' from 'langchain.chains' (c:\Users\ffollonier\AppData\Local\anaconda3\envs\basiclc\Lib\site-packages\langchain\chains\__init__.py)

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter

loader = DirectoryLoader('../data/qna/', glob="*.txt", loader_cls=TextLoader)

documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Next, let's ingest them into FAISS so we can efficiently query our embeddings:

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

Now let's create a chain that can do the whole chat on our embedding database:

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

Now let's ask a question:

In [ ]:
chat_history = []
query = "what is azure openai service?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

Azure OpenAI is a service that provides REST API access to OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service is available in East US, South Central US, and West Europe regions. Additionally, Azure OpenAI offers private networking, regional availability, and responsible AI content filtering.


We can use this to easy implement chat conversations:

In [ ]:
chat_history = []

query = "what is Azure OpenAI Service?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

chat_history = [(query, result["answer"])]
query = "Which regions does the service support?"
result = qa({"question": query, "chat_history": chat_history})
print("Question:", query)
print("Answer:", result["answer"])

Question: what is Azure OpenAI Service?
Answer: Azure OpenAI Service is a cloud-based service that provides REST API access to OpenAI's language models, including GPT-3, Codex, and Embeddings model series. These models can be used for various tasks such as content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or a web-based interface in the Azure OpenAI Studio. The service is built on Microsoft Azure and offers features such as virtual network support, managed identity, and responsible AI content filtering. However, access to the service is currently limited due to high demand and Microsoft's commitment to responsible AI.
Question: Which regions does the service support?
Answer: The Azure OpenAI Service is currently available in the East US, South Central US, and West Europe regions.
